In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import sys
from unicodedata import category
import pandas as pd
punctuation_chars_numbers =  set([chr(i) for i in range(sys.maxunicode) 
                             if category(chr(i)).startswith("P") or category(chr(i)).startswith("Nd")]) # Добавили кучу пунктуационных знаков и чисел
print(punctuation_chars_numbers)

sw = set(stopwords.words('russian'))

from pymystem3 import Mystem
mystem_analyzer = Mystem(entire_input=False)

def MyTokenizer(sentence):
    tokens = mystem_analyzer.lemmatize(sentence.lower())
    tokens = [x for x in tokens if (len(set(x) & punctuation_chars_numbers) == 0 and len(set([x]) & sw) == 0 and len(x)>2)]
    return ' '.join(tokens)
# Использую mystem для лемматицазиции и удаляю большой список лишних символов и слов из текста с помощью пересечения(&) сетов.
# Пересечение гораздо быстрее работает, чем поэлементная проверка с помощью if x in array

In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/home/seara/Desktop/Colab Notebooks/homework/hw06-text/train.csv')
test = pd.read_csv('/home/seara/Desktop/Colab Notebooks/homework/hw06-text/test.csv')

In [ ]:
train.fillna("", inplace=True)
test.fillna("", inplace=True)
# Заполнение пропусков

In [ ]:
test["desctitletokens"] = test["title"] + " " + test["description"]
train["desctitletokens"] = train["title"] + " " + train["description"]
test.drop(["title","description"], axis=1, inplace=True)
train.drop(["title","description"], axis=1, inplace=True)
# Создание единой колонки и дроп лишних

In [ ]:
%%time
train["desctitletokens"] = train["desctitletokens"].apply(MyTokenizer)
test["desctitletokens"] = test["desctitletokens"].apply(MyTokenizer)
# Применение токенизатора ко всему датасету, чтобы потом можно было сразу токенизированный датафрейм подгружать из pickle файла

In [ ]:
import pickle
with open('traintokenized', 'wb') as data:
    pickle.dump(train, data)

with open('testtokenized', 'wb') as data:
    pickle.dump(test, data)

In [ ]:
import pickle
import pandas as pd
with open(
    "traintokenized", "rb"
) as data:
    train = pickle.load(data)
with open("testtokenized", "rb") as data:
    test = pickle.load(data)

In [ ]:
def beauty(row):
    text = ''
    if row["Category"] == 86:
        if 'коммерческий' in row["desctitletokens"].split(' '):
            text = " ".join(row["desctitletokens"].split(' ')[0:3])
        else:
            text = " ".join(row["desctitletokens"].split(' ')[0:2])
    else:
        text = row["desctitletokens"]
    return text
# Чуть изменим текст 86 категории в тренировочном датасете для лучших результатов
# Не нашел способов как-то вручную улучшить тренировочные данные для остальных категорий

In [ ]:
%%time
train["desctitletokens"] = train.apply(beauty, axis=1)

In [ ]:
X = train["desctitletokens"]
y = train["Category"]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [ ]:
%%time
clf = Pipeline([("vec", TfidfVectorizer(ngram_range=(1, 2))), ("clf", SGDClassifier(n_jobs=-1, alpha=0.0000002,tol=1e-4))])
clf.fit(X,y)
# Обучаем на всей выборке
# Параметры для SGD были подобраны с помощью GRIDSEARCH и многочисленных тестов

In [ ]:
pred = clf.predict(test['desctitletokens'])
data = {'Id':test["itemid"],
        'Category':pred}

df = pd.DataFrame(data)
df.to_csv("finalattempt.csv",index=False)
# Загружаем на kaggle